In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

# Arenstorf orbit

In [ ]:
class arenstorf_model: 
 
    def __init__(self, mu): 
        self.mu = mu 
 
    def fcn(self, t, y) : 
        y1,y2,y3,y4 = y 
        mu = self.mu 
        r1 = np.sqrt((y1+mu)*(y1+mu) + y2*y2) 
        r2 = np.sqrt((y1-1+mu)*(y1-1+mu) + y2*y2) 
        y1_dot = y3 
        y2_dot = y4 
        y3_dot = y1 + 2*y4 - (1-mu)*(y1+mu)/(r1*r1*r1) - mu*(y1 - 1 + mu)/(r2*r2*r2) 
        y4_dot = y2 - 2*y3 - (1-mu)*y2/(r1*r1*r1) - mu*y2/(r2*r2*r2) 
        return np.array([y1_dot, y2_dot, y3_dot, y4_dot])

    def V_q(self, q): 
        mu = self.mu 
        q1 = q[0]  
        q2 = q[1]  
        
        # not rhe same r1, r2 than fcn
        r1 = np.sqrt((q1-1+mu)*(q1-1+mu) + q2*q2) 
        r2 = np.sqrt((q1+mu)*(q1+mu) + q2*q2) 
         
        ##Hq1 = q1 + 2*p2 - (1-mu)*(q1+mu)/(r1*r1*r1) - mu*(q1 - 1 + mu)/(r2*r2*r2) 
        ##Hq2 = q2 - 2*p1 - (1-mu)*q2/(r1*r1*r1) - mu*q2/(r2*r2*r2) 
        V_q1 = -q1 + (1-mu)*(q1+mu)/(r2*r2*r2) + mu*(q1-1+ mu)/(r1*r1*r1) 
        V_q2 = -q2 + (1-mu)*q2/(r2*r2*r2) + mu*q2/(r1*r1*r1) 
 
        return np.array((V_q1, V_q2)) 
 
    def Fp(self, t, p): 
        p1 = p[0]  
        p2 = p[1] 
 
        w = 2. 
         
        Fp1 = t*p1 + ((1-np.cos(w*t))/(w*w))*2*p2 + ((np.sin(w*t) - w*t)/(w*w*w))*4*p1 
        Fp2 = t*p2 - ((1-np.cos(w*t))/(w*w))*2*p1 + ((np.sin(w*t) - w*t)/(w*w*w))*4*p2 
         
        return np.array((Fp1, Fp2)) 
 
    def expp(self, t, p): 
        p1 = p[0]  
        p2 = p[1]  
 
        w = 2. 
         
        expp1 = p1 + (np.sin(w*t)/w)*2*p2 - 2*(np.sin(0.5*w*t)/w)*(np.sin(0.5*w*t)/w)*4*p1 
        expp2 = p2 - (np.sin(w*t)/w)*2*p1 - 2*(np.sin(0.5*w*t)/w)*(np.sin(0.5*w*t)/w)*4*p2 
         
        return np.array((expp1, expp2)) 

    def hamiltonian(self, y):
    
        mu = self.mu
        nt = y.shape[0]
        neq = y.shape[1]
        q = y[:,0:neq//2]
        p = y[:,neq//2:neq]
        ham = np.zeros(nt)

        for i in range(nt):
            
            q1 = q[i,0]
            q2 = q[i,0]
            r1 = np.sqrt((q1-1+mu)*(q1-1+mu) + q2*q2) 
            r2 = np.sqrt((q1+mu)*(q1+mu) + q2*q2) 
            ham[i] = 0.5*np.dot(p[i],p[i]) - 0.5*(q1*q1+q2*q2) - mu/r1 - (1-mu)/r2

        return ham

## Integration

In [ ]:
#################################################################
def scovel(tini, tend, nt, yini, V_q, Fp, expp):
    
    dt = (tend-tini) / (nt-1) 

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((nt, neq))
    y[0] = yini_array

    for it in range(nt-1):
        q_n = y[it, 0:neq//2]
        p_n = y[it, neq//2:neq]

        p_np05 = p_n - (dt/2)*V_q(q_n)
        q_np1  = q_n + Fp(dt, p_np05)
        p_np1  = expp(dt, p_np05) - (dt/2)*V_q(q_np1)

        y[it+1, 0:neq//2] = q_np1
        y[it+1, neq//2:neq] = p_np1

    return y

#################################################################
def optimized_815_scov(tini, tend, nt, yini, V_q, Fp, expp): 
    
    def scov(tini, tend, nt, yini, V_q, Fp, expp):
        dt = (tend-tini) / (nt-1)
        yini_array = np.array(yini)
        neq = yini_array.size
        y = yini_array
        for it in range(nt-1):
            q_n = y[0:neq//2]
            p_n = y[neq//2:neq]
            p_np05 = p_n - (dt/2)*V_q(q_n)
            q_np1  = q_n + Fp(dt, p_np05)
            p_np1  = expp(dt, p_np05) - (dt/2)*V_q(q_np1)
            y[0:neq//2] = q_np1
            y[neq//2:neq] = p_np1
        return y
    
    dt = (tend-tini) / (nt-1) 

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((nt, neq))
    y[0] = yini_array

    nstep = 15
    gamma = np.zeros(nstep+1)
    gamma[0]  =  0.
    gamma[1]  =  0.74167036435061295344822780
    gamma[2]  = -0.40910082580003159399730010
    gamma[3]  =  0.19075471029623837995387626
    gamma[4]  = -0.57386247111608226665638773
    gamma[5]  =  0.29906418130365592384446354
    gamma[6]  =  0.33462491824529818378495798
    gamma[7]  =  0.31529309239676659663205666
    gamma[8]  = -0.79688793935291635401978884
    gamma[9]  = gamma[7]
    gamma[10] = gamma[6]
    gamma[11] = gamma[5]
    gamma[12] = gamma[4]
    gamma[13] = gamma[3]
    gamma[14] = gamma[2]
    gamma[15] = gamma[1]
                          
    ytmp = y[0]
    for it in range(nt-1):
 
        for istep in range(nstep):

            ti = 0.0
            te = gamma[istep+1]*dt
            ytmp = scov(ti, te, 2, ytmp, V_q, Fp, expp)

        y[it+1] = ytmp

    return y

In [ ]:
qini = np.array((0.994, 0.))
pini = np.array((0., -2.00158510637908252240537862224))
yini = np.concatenate((qini, pini))

tini = 0.
tend = 1*18. #18 roughly corresponds to 1 period

am = arenstorf_model(mu=0.012277471)
fcn  = am.fcn 
V_q = am.V_q
Fp   = am.Fp
expp = am.expp

nt_sco = 50001
ysco = scovel(tini, tend, nt_sco, yini, V_q, Fp, expp)

nt_o815 = 10001
yo815 = optimized_815_scov(tini, tend, nt_o815, yini, V_q, Fp, expp)

tol = 1.e-6
sol_rk45 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)

tol = 1.e-6
sol_dopri853 = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=tol, atol=tol)

# plotting

# marker style
nt_sco_display = nt_sco//50
marker01=dict(size=5, symbol='x-thin', line=dict(width=1, color='#d62728'), maxdisplayed=nt_sco_display)
marker02=dict(size=5, symbol='x-thin', line=dict(width=1, color='#1f77b4'))
nt_o815_display = nt_o815//10
marker03=dict(size=5, symbol='x-thin', line=dict(width=1, color='#9467bd'), maxdisplayed=nt_o815_display)
marker04=dict(size=5, symbol='x-thin', line=dict(width=1, color='#8c564b'))

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=ysco[:,0], y=ysco[:,1], mode='markers', marker=marker01, name='Scovel'))
fig_sol.add_trace(go.Scatter(x=sol_rk45.y[0], y=sol_rk45.y[1], mode='markers', marker=marker02, name='RK45'))
fig_sol.add_trace(go.Scatter(x=sol_rk45.y[0], y=sol_rk45.y[1], mode='markers', marker=marker03, name='Optimized 8-15'))
fig_sol.add_trace(go.Scatter(x=sol_dopri853.y[0], y=sol_dopri853.y[1], mode='markers', marker=marker04, name='Dopri 853'))
fig_sol.show()